In [1]:
import pandas as pd
from statsforecast import StatsForecast
from statsforecast.models import Naive
import matplotlib.pyplot as plt
import statsforecast as sf
from utilsforecast.plotting import plot_series
import numpy as np
from sktime.forecasting.ets import AutoETS
from sktime.forecasting.base import ForecastingHorizon
from sklearn.metrics import mean_absolute_percentage_error
from sktime.forecasting.arima import AutoARIMA
from sktime.forecasting.fbprophet import Prophet
from sktime.forecasting.model_selection import ForecastingGridSearchCV, ExpandingWindowSplitter
from sktime.performance_metrics.forecasting import MeanAbsolutePercentageError

import warnings
warnings.filterwarnings("ignore")

import logging
logging.getLogger("cmdstanpy").setLevel(logging.WARNING)
logging.disable(logging.CRITICAL)

df = pd.read_excel('Downloads/CMO-Historical-Data-Monthly.xlsx', sheet_name='Monthly Prices')
df_clean = df.drop([0, 1, 2, 3, 4]) 
new_columns = df.iloc[3, 1:].tolist()  
df_clean.columns = ['Month'] + new_columns  
df_clean['Month'] = pd.to_datetime(df_clean['Month'], format='%YM%m')
df_clean.set_index('Month', inplace=True)
df_clean = df_clean.sort_index()
df_clean = df_clean.dropna(axis=1, how='all')

df = df_clean
df.index = df.index.to_period('M')
for col in df.columns:
    df[col] = pd.to_numeric(df[col], errors='coerce')
df

,"Crude oil, average","Crude oil, Brent","Crude oil, Dubai","Crude oil, WTI","Coal, Australian","Coal, South African **","Natural gas, US","Natural gas, Europe","Liquefied natural gas, Japan",Natural gas index,...,Aluminum,"Iron ore, cfr spot",Copper,Lead,Tin,Nickel,Zinc,Gold,Platinum,Silver
Month,,,,,,,,,,,,,,,,,,,,,
1960-01,1.630000,1.630,1.63,NaN,NaN,NaN,0.1400,0.404774,NaN,NaN,...,511.471832,11.42,715.40,206.10,2180.40,1631.00,260.80,35.27,83.50,0.9137
1960-02,1.630000,1.630,1.63,NaN,NaN,NaN,0.1400,0.404774,NaN,NaN,...,511.471832,11.42,728.19,203.70,2180.40,1631.00,244.90,35.27,83.50,0.9137
1960-03,1.630000,1.630,1.63,NaN,NaN,NaN,0.1400,0.404774,NaN,NaN,...,511.471832,11.42,684.94,210.30,2173.80,1631.00,248.70,35.27,83.50,0.9137
1960-04,1.630000,1.630,1.63,NaN,NaN,NaN,0.1400,0.404774,NaN,NaN,...,511.471832,11.42,723.11,213.60,2178.20,1631.00,254.60,35.27,83.50,0.9137
1960-05,1.630000,1.630,1.63,NaN,NaN,NaN,0.1400,0.404774,NaN,NaN,...,511.471832,11.42,684.75,213.40,2162.70,1631.00,253.80,35.27,83.50,0.9137
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2024-11,72.291667,74.395,72.79,69.69,142.12,106.82,2.1045,13.926400,12.822731,100.448776,...,2582.190000,100.50,9075.73,1987.53,29654.89,15723.06,3004.20,2651.13,965.53,31.0940
2024-12,72.311000,73.833,73.31,69.79,129.81,105.47,3.0229,13.856621,12.640616,111.146139,...,2541.020000,102.21,8916.32,1990.43,28864.99,15444.89,3034.16,2648.01,937.88,30.7640
2025-01,78.162000,79.206,80.14,75.14,118.60,103.28,4.0991,14.663512,13.193106,128.425081,...,2573.400000,99.58,8991.41,1921.36,29612.36,15394.14,2818.96,2709.69,949.23,30.4110


In [2]:
def get_mape(ts_num, forecaster):
    series = df.iloc[:, ts_num].dropna()
    train = series[series.index < test_start]
    test = series[(series.index >= test_start) & (series.index <= test_end)]
    forecaster.fit(train)
    fh = ForecastingHorizon(test.index, is_relative=False)
    y_pred = forecaster.predict(fh)
    return mean_absolute_percentage_error(test, y_pred), y_pred

ARIMA = AutoARIMA(
    sp=12,
    seasonal=False,
    stepwise=False,       
    n_jobs=-1,            
    trace=False,
    error_action='ignore',
    suppress_warnings=True
)

SARIMA = AutoARIMA(
    sp=12,
    seasonal=True,
    stepwise=False,      
    n_jobs=-1,          
    trace=False,
    error_action='ignore',
    suppress_warnings=True
)

ETS = AutoETS(
    auto=True, 
    sp=12, 
    n_jobs=-1)

def get_mape_ets(ts_num):
    res = get_mape(ts_num, ETS)
    return res[0], 'ETS', res[1]
def get_mape_arima(ts_num):
    res = get_mape(ts_num, ARIMA)
    return res[0], 'ARIMA', res[1]
def get_mape_sarima(ts_num):
    res = get_mape(ts_num, SARIMA)
    return res[0], 'SARIMA', res[1]

In [3]:
def get_mape_prophet(ts_num):
    series = df.iloc[:, ts_num].dropna()
    train = series[series.index < test_start]
    test = series[(series.index >= test_start) & (series.index <= test_end)]

    param_grid = {
        'seasonality_mode': ['additive', 'multiplicative'],
    }
    cv = ExpandingWindowSplitter(
        initial_window=24, 
        step_length=12,   
        fh=[1,2,3,4,5,6,7,8,9,10,11,12] 
    )
    mape = MeanAbsolutePercentageError()
    grid_search = ForecastingGridSearchCV(
        forecaster=Prophet(),
        param_grid=param_grid,
        cv=cv,
        scoring=mape
    )
    grid_search.fit(train)
    forecaster = Prophet(
        seasonality_mode=grid_search.best_params_['seasonality_mode'], 
        n_changepoints=int(len(train) / 12),  
        yearly_seasonality=True
    )
    
    forecaster.fit(train)
    fh = ForecastingHorizon(test.index, is_relative=False)
    y_pred = forecaster.predict(fh)
    return mean_absolute_percentage_error(test, y_pred), 'PROPHET', y_pred

In [4]:
def get_mape_const_last(ts_num):
    series = df.iloc[:, ts_num].dropna()
    train = series[series.index < test_start]
    test = series[(series.index >= test_start) & (series.index <= test_end)]

    periods = len(series[(series.index >= test_start) & (series.index <= test_end)])
    last_obs = train[train.index.max()]
    period_index = pd.period_range(start=test_start, periods=periods, freq='M')
    y_pred = pd.Series([last_obs]*periods, index=period_index)
    return mean_absolute_percentage_error(test, y_pred), 'const_last', y_pred
    
def get_mape_const_mean(ts_num):
    series = df.iloc[:, ts_num].dropna()
    train = series[series.index < test_start]
    test = series[(series.index >= test_start) & (series.index <= test_end)]

    periods = len(series[(series.index >= test_start) & (series.index <= test_end)])
    tr = pd.Period(test_start, freq='M') - periods
    mean = train.loc[tr:tr + periods].mean()
    period_index = pd.period_range(start=test_start, periods=periods, freq='M')
    y_pred = pd.Series([mean]*periods, index=period_index)
    return mean_absolute_percentage_error(test, y_pred), 'const_mean', y_pred

In [5]:
funcs = [get_mape_ets, get_mape_arima, get_mape_sarima, get_mape_prophet, get_mape_const_last, get_mape_const_mean]

In [6]:
#предсказываем 5 лет 2020-2024

In [7]:
test_start = '2020-01'
test_end = '2024-12'

In [9]:
results = []
for ts_num in range(len(df.columns)):
    verdicts = {}
    verdicts['ts_num'] = ts_num
    verdicts['ts_name'] = df.columns[ts_num]

    series = df.iloc[:, ts_num].dropna()
    train = series[series.index < test_start]
    test = series[(series.index >= test_start) & (series.index <= test_end)]
    verdicts['y_test'] = test
    verdicts['y_all_series'] = series

    for func in funcs:
        try:
            res = func(ts_num)
            model_name = res[1]
            verdicts[model_name] = res[0]
            verdicts[model_name + '_pred'] = res[2]
        except:
            pass
    results.append(verdicts)
    print(ts_num)
    print(verdicts.keys())

metrics_df = pd.DataFrame(results)
metrics_df.to_csv(test_start + '_' + test_end + '_predicts.csv', index=False)

def can_convert_to_float(series):
    try:
        pd.to_numeric(series)
        return True
    except Exception:
        return False

cols_to_keep = [col for col in metrics_df.columns if can_convert_to_float(metrics_df[col])]
metrics_df_float = metrics_df[cols_to_keep]
metrics_df_float = metrics_df_float.drop(columns=['ts_num'])

stats = pd.DataFrame({
    'mean': metrics_df_float.mean(),
    'median': metrics_df_float.median(),
    'max': metrics_df_float.max()
})

stats = 100 * stats
stats.columns.name = 'Model'
stats.index.name = 'Aggregate'
stats = stats.round(2)
stats

0
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])
1
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])
2
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])
3
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])
4
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SAR

/opt/anaconda3/lib/python3.12/site-packages/sktime/performance_metrics/forecasting/_classes.py:660: UserWarning: y_pred and y_true do not have the same row index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/sktime/performance_metrics/forecasting/_classes.py:660: UserWarning: y_pred and y_true do not have the same row index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/sktime/performance_metrics/forecasting/_classes.py:660: UserWarning: y_pred and y_true do not have the same row index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/sktime/performance_metrics/forecasting/_classes.py:660: UserWarning: y_pred and y_true do not have the same row index. This may indicate inc

32
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])
33
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])


/opt/anaconda3/lib/python3.12/site-packages/sktime/performance_metrics/forecasting/_classes.py:660: UserWarning: y_pred and y_true do not have the same row index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/sktime/performance_metrics/forecasting/_classes.py:660: UserWarning: y_pred and y_true do not have the same row index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/sktime/performance_metrics/forecasting/_classes.py:660: UserWarning: y_pred and y_true do not have the same row index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/sktime/performance_metrics/forecasting/_classes.py:660: UserWarning: y_pred and y_true do not have the same row index. This may indicate inc

34
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])
35
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])
36
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])
37
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])
38
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA',

/opt/anaconda3/lib/python3.12/site-packages/statsmodels/base/model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "


46
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])
47
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])
48
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])
49
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])


/opt/anaconda3/lib/python3.12/site-packages/sktime/performance_metrics/forecasting/_classes.py:660: UserWarning: y_pred and y_true do not have the same row index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/sktime/performance_metrics/forecasting/_classes.py:660: UserWarning: y_pred and y_true do not have the same row index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/sktime/performance_metrics/forecasting/_classes.py:660: UserWarning: y_pred and y_true do not have the same row index. This may indicate incorrect objects passed to the metric. Indices of y_true will be used for y_pred.
  warn(
/opt/anaconda3/lib/python3.12/site-packages/sktime/performance_metrics/forecasting/_classes.py:660: UserWarning: y_pred and y_true do not have the same row index. This may indicate inc

50
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])
51
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])
52
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])
53
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA', 'SARIMA_pred', 'PROPHET', 'PROPHET_pred', 'const_last', 'const_last_pred', 'const_mean', 'const_mean_pred'])
54
dict_keys(['ts_num', 'ts_name', 'y_test', 'y_all_series', 'ETS', 'ETS_pred', 'ARIMA', 'ARIMA_pred', 'SARIMA',

Model,mean,median,max
Aggregate,,,
ETS,23.20,23.27,68.10
ARIMA,22.07,22.20,67.40
SARIMA,22.27,22.15,67.40
PROPHET,27.34,24.02,79.22
const_last,23.96,24.60,67.96
const_mean,24.81,24.61,68.87
